In [1]:
pip install pandas

In [30]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd


def transform_data(input_file_path, output_file_path):
    # Read the input CSV file
    df = pd.read_csv(input_file_path)

    # Sort data by Employee Code and Date of Joining
    df.sort_values(by=['Employee Code', 'Date of Joining'], inplace=True)

    # Derive 'End Date' for each record
    df['End Date'] = df.groupby('Employee Code')['Date of Joining'].shift(-1)
    df['End Date'] = df['End Date'].fillna(pd.to_datetime(df['Date of Exit'], errors='coerce')).fillna(pd.to_datetime('2100-01-01'))

    # Fill missing values with the most recent past record
    df.fillna(method='ffill', inplace=True)

    # Create a new DataFrame for historical records
    historical_df = pd.DataFrame(columns=[
        'Employee Code', 'Manager Employee Code', 'Last Compensation',
        'Compensation', 'Last Pay Raise Date', 'Variable Pay', 'Tenure in Org',
        'Performance Rating', 'Engagement Score', 'Effective Date', 'End Date'
    ])

    # Iterate through each row and expand the records
    for index, row in df.iterrows():
        period_dates = pd.date_range(start=row['Date of Joining'], end=row['End Date'], freq='D')
        for date in period_dates:
            historical_row = {
                'Employee Code': row['Employee Code'],
                'Manager Employee Code': row['Manager Employee Code'],
                'Last Compensation': row['Compensation'],
                'Compensation': row['Compensation 1'] if date == pd.to_datetime(row['Compensation 1 date'], errors='coerce') else row['Compensation 2'],
                'Last Pay Raise Date': pd.to_datetime(row['Compensation 1 date'], errors='coerce') if date == pd.to_datetime(row['Compensation 1 date'], errors='coerce') else pd.to_datetime(row['Compensation 2 date'], errors='coerce'),
                'Variable Pay': 0,  # Assuming no variable pay in the provided data
                'Tenure in Org': 0,  # You might need to calculate this based on the actual data
                'Performance Rating': row['Review 1'] if date == pd.to_datetime(row['Review 1 date'], errors='coerce') else row['Review 2'],
                'Engagement Score': row['Engagement 1'] if date == pd.to_datetime(row['Engagement 1 date'], errors='coerce') else row['Engagement 2'],
                'Effective Date': date,
                'End Date': row['End Date'],
            }
            historical_df = historical_df.append(historical_row, ignore_index=True)

    # Save the output to a new CSV file
    historical_df.to_csv(output_file_path, index=False)

if __name__ == "__main__":
    input_file_path = '/content/input.csv'
    output_file_path = '/content/output.csv'

    transform_data(input_file_path, output_file_path)

In [31]:
historical_df

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date,End Date


In [32]:
historical_row

Employee Code                                        1
Manager Employee Code                              NaN
Date of Joining                             2021-01-01
Date of Exit                                       NaN
Compensation                                     20000
Compensation 1                                     NaN
Compensation 1 date                                NaN
Compensation 2                                     NaN
Compensation 2 date                                NaN
Review 1                                           NaN
Review 1 date                                      NaN
Review 2                                           NaN
Review 2 date                                      NaN
Engagement 1                                       NaN
Engagement 1 date                                  NaN
Engagement 2                                       NaN
Engagement 2 date                                  NaN
End Date                           2100-01-01 00:00:00
Date      

In [33]:

    transform_data

<function __main__.transform_data(input_file_path, output_file_path)>

In [34]:
period_dates

DatetimeIndex(['1970-01-01 00:00:00.000000001',
               '1970-01-02 00:00:00.000000001',
               '1970-01-03 00:00:00.000000001',
               '1970-01-04 00:00:00.000000001',
               '1970-01-05 00:00:00.000000001',
               '1970-01-06 00:00:00.000000001',
               '1970-01-07 00:00:00.000000001',
               '1970-01-08 00:00:00.000000001',
               '1970-01-09 00:00:00.000000001',
               '1970-01-10 00:00:00.000000001',
               ...
               '2020-12-22 00:00:00.000000001',
               '2020-12-23 00:00:00.000000001',
               '2020-12-24 00:00:00.000000001',
               '2020-12-25 00:00:00.000000001',
               '2020-12-26 00:00:00.000000001',
               '2020-12-27 00:00:00.000000001',
               '2020-12-28 00:00:00.000000001',
               '2020-12-29 00:00:00.000000001',
               '2020-12-30 00:00:00.000000001',
               '2020-12-31 00:00:00.000000001'],
              dtype=

In [35]:
df.sort_values

<bound method DataFrame.sort_values of    Employee Code  Manager Employee Code Date of Joining Date of Exit  \
0              1                    NaN      2021-01-01          NaN   
1              2                    1.0      2021-01-01          NaN   
2              3                    1.0      2021-01-01   2023-12-31   

   Compensation  Compensation 1 Compensation 1 date  Compensation 2  \
0         20000             NaN                 NaN             NaN   
1         20000         10000.0          2022-01-01         20000.0   
2         20000         10000.0          2022-01-01         20000.0   

  Compensation 2 date  Review 1 Review 1 date  Review 2 Review 2 date  \
0                 NaN       NaN           NaN       NaN           NaN   
1          2023-01-01       9.0    2021-06-01       9.5    2022-06-01   
2          2023-01-01       9.0    2021-06-01       9.5    2022-06-01   

   Engagement 1 Engagement 1 date  Engagement 2 Engagement 2 date   End Date  
0           NaN               NaN           NaN               NaN 2100-01-01  
1           4.0        2021-03-01           5.0        2022-03-01 2100-01-01  
2           4.0        2021-03-01           5.0        2022-03-01 2100-01-01  >